# Sta rade botovi po nasem sajtu i koliko ih je

## Definisemo nasu semu jer CSV nije imao header

nakon detaljnijeg prolazenja kroz data set video sam da se pojavljuje action 142 na 2 mesta iz nekog razloga i on mora da se excluduje iz naseg data seta jer sajtovi nemaju akciju 142 vec samo edit , new , categorize and log



In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType

# Define the schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("action", StringType(), True),
    StructField("title", StringType(), True),
    StructField("comment", StringType(), True),
    StructField("user", StringType(), True),
    StructField("bot", BooleanType(), True)
    # Add as many columns as you have in your CSV
])

# Read the CSV file
df = spark.read.format("csv").option("header", "true").schema(schema).load("dbfs:/FileStore/shared_uploads/pstamenic7721rn@raf.rs/output0-1.csv")

df = df.filter(df.action != "142")
# Show the DataFrame
df.show()

+----------+----------+--------------------+--------------------+----------------+-----+
|        id|    action|               title|             comment|            user|  bot|
+----------+----------+--------------------+--------------------+----------------+-----+
|2133549229|      edit|           Q36267081|/* wbsetreference...|          Cewbot| true|
|2133549228|      edit|           Q64923313|/* wbcreateclaim-...|         Mitsjol|false|
|1734135754|categorize|Category:Players ...|[[:Willie Adams (...|Dominus Moravian|false|
|1734135755|categorize|Category:WikiProj...|[[:Talk:Pfandbrie...|  Qwerfjkl (bot)| true|
|2133549230|      edit|           Q61043507|/* wbsetlabel-add...|         StultuS|false|
|  76240560|      edit|   טיוטה:מבצר מסילחה|/* קישורים חיצוני...|  Danny Gershoni|false|
|2133549232|      edit|          Q116332994|/* wbeditentity-u...|     Dcirovicbot| true|
|1734135756|      edit|Talk:New York Sta...|[[User:Cewbot/log...|          Cewbot| true|
|2133549231|      edi

## Veoma nam je bitno da vidimo koliko usera koristi koju funkcionalnost na nasem sajtu kao i koji je odnos botova prema pravim userima pri tim funkcionalnostima

Ovde mozemo videti sta se desava ispod haube, koliko je aktivnih korisnika na nasem sajtu a koliko je botova

Ukoliko u nekom vecem vremenskom periodu primetimo da se broj botova za logovanje smanjuje to moze predstavljati da je jedan od nasih botova prestao sa radom dok ukoliko porast u broju botova za logovanje bez da smo mi dodavalai dodatne botove to moze znaciti da neko prisluskuje desavanja na nasem sajtu.

Takodje mozemo videti i koliko je contenta na nasem sajtu napisano od strane botova u poredjenju sa ljudima

In [0]:
from pyspark.sql.functions import countDistinct

# Assuming 'df' is your Spark DataFrame

# Count distinct bot users aggregated by action
bot_user_count = df.filter(df["bot"] == True).groupBy("action").agg(countDistinct("user").alias("distinct_bot_users"))

# Count distinct non-bot users aggregated by action
non_bot_user_count = df.filter(df["bot"] == False).groupBy("action").agg(countDistinct("user").alias("distinct_non_bot_users"))

# Join the two results on the action column
action_user_counts = bot_user_count.join(non_bot_user_count, "action")

# Show the result
action_user_counts.show()

+----------+------------------+----------------------+
|    action|distinct_bot_users|distinct_non_bot_users|
+----------+------------------+----------------------+
|       new|                12|                   264|
|       log|                 7|                   272|
|      edit|                82|                  2509|
|categorize|                37|                   684|
+----------+------------------+----------------------+



In [0]:
from pyspark.sql.functions import col, sum as _sum, count as _count

df = df.withColumn("bot", col("bot").cast("integer"))
# Group by 'action' and calculate bot and user counts
action_df = df.groupBy("action").agg(
    _sum("bot").alias("bot_count"),
    (_count("*") - _sum("bot")).alias("user_count")
)

# Calculate the ratio of bots to users
action_df = action_df.withColumn("bot_to_user_ratio", col("bot_count") / col("user_count"))

# Show the results
action_df.show()


+----------+---------+----------+--------------------+
|    action|bot_count|user_count|   bot_to_user_ratio|
+----------+---------+----------+--------------------+
|       new|       35|       590|0.059322033898305086|
|       log|      825|       644|   1.281055900621118|
|      edit|     9656|     11522|   0.838048949835098|
|categorize|     5643|      8089|  0.6976140437631351|
+----------+---------+----------+--------------------+



## Kako se dobijeni podaci mogu koristiti na duze vremenske periode
Ovde je simulirano deljenje velikog dataset na 2 manja 70-30 koji bi predstavljali koliki procenat poruka po kategoriji je napisao bot
Ukoliko bi se ovaj procenat koji imamo od duzeg vremena vec naglo povecao na primer u poslednjem data set koji smo stavili da se testira
To bi moglo znaciti da neko pokusava da napada nas sajt sa botovima iz raznih razloga (uklanjanje politickih materijala, propaganda i sl)
<br>
<br>
u ovom primeru dozvoljeno je odstupanje od 5% od prethodne prosecne vrednosti

In [0]:
from pyspark.sql.functions import col, count, when

# Split the DataFrame into training and testing sets
train_df, test_df = df.randomSplit([0.7, 0.3])

# Calculate total message counts for training and testing DataFrames
total_train_msgs = train_df.count()
total_test_msgs = test_df.count()

# Aggregate data for training DataFrame with percentage calculation
aggregated_train_df = train_df.groupBy("action").agg(
    (count(when(col("bot") == True, True)) / total_train_msgs * 100).alias("bot_activity_percent_train")
)
aggregated_train_df.show()

# Aggregate data for testing DataFrame with percentage calculation
aggregated_test_df = test_df.groupBy("action").agg(
    (count(when(col("bot") == True, True)) / total_test_msgs * 100).alias("bot_activity_percent_test")
)
aggregated_test_df.show()

# Join the aggregated data on 'action'
joined_df = aggregated_train_df.join(aggregated_test_df, "action")

# Calculate the percentage change in bot activity
joined_df = joined_df.withColumn("perc_change", 
                                 (col("bot_activity_percent_test") - col("bot_activity_percent_train")) / col("bot_activity_percent_train") * 100)

joined_df.show()

# Filter where the increase is 5% or more
increased_df = joined_df.filter(col("perc_change") >= 5)

# Show or alert the user
increased_df.show()

+----------+--------------------------+
|    action|bot_activity_percent_train|
+----------+--------------------------+
|       new|       0.08823293494826343|
|       log|        2.2379080773241355|
|      edit|        27.063447501403704|
|categorize|        15.966150637683485|
+----------+--------------------------+

+----------+-------------------------+
|    action|bot_activity_percent_test|
+----------+-------------------------+
|       new|       0.1236564253781033|
|       log|       2.4826405402834584|
|      edit|       27.651479121088173|
|categorize|       15.808998382954437|
+----------+-------------------------+

+----------+--------------------------+-------------------------+-------------------+
|    action|bot_activity_percent_train|bot_activity_percent_test|        perc_change|
+----------+--------------------------+-------------------------+-------------------+
|       new|       0.08823293494826343|       0.1236564253781033| 40.147695926255786|
|       log|        2.

## Takodje je moguce proveriti i prosecnu duzinu poruke od strane botova naspram korisnika po oblasti

ukoliko neka od poruka drasticno premasi prosek mozda nije lose da se flaguje da je moderator proveri ili u slucaju da duzina logova se drasticno promeni moze naznaciti da imamo problem sa jednim od nasih botova.

takodje mozemo zakljuciti da botovi obicno pisu krace nove poruke dok korisnici obicno pisu duze nove poruke tako da i po duzini poruka mozemo nekoga flagovati da je potencijalni bot account

In [0]:
from pyspark.sql.functions import length, avg

# Add a new column for comment length
df_with_length = df.withColumn("comment_length", length(df["comment"]))

# Calculate average comment length for bots
bot_avg_comment_length = df_with_length.filter(df_with_length["bot"] == True).groupBy("action").agg(avg("comment_length").alias("avg_len_bot_comments"))

# Calculate average comment length for non-bots
non_bot_avg_comment_length = df_with_length.filter(df_with_length["bot"] == False).groupBy("action").agg(avg("comment_length").alias("avg_len_non_bot_comments"))

# Join the two DataFrames on the action column
comparison_df = bot_avg_comment_length.join(non_bot_avg_comment_length, "action")

# Show the result
comparison_df.show()

+----------+--------------------+------------------------+
|    action|avg_len_bot_comments|avg_len_non_bot_comments|
+----------+--------------------+------------------------+
|       new|   36.42857142857143|       118.0442930153322|
|       log|  101.81074481074481|      57.323232323232325|
|      edit|  125.53744174003107|       74.23206167226725|
|categorize|  125.81694134325713|       76.17999752750649|
+----------+--------------------+------------------------+



## Najcesce koriscenje reci kod botova i kod korisnika
mozemo i proveriti koje su najcesce koriscenje reci od strane botova naspram regularnih korisnika.
potrebno je dodatno urediti dataset
nazalost zbog limitacije databricks nije bilo moguce pokrenuti uklanjanje svih reci koje nisu imale preterano smisla da se nadju u listi kao na primer pocetak i kraj zavrsavanja komentara sto smo dobili sa api /* *i* */


In [0]:
from pyspark.sql.functions import explode, col
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace

df = df.filter(col("comment").isNotNull() & (col("comment").cast("string") == col("comment")))


# # Remove '/*' and '*/'
# df = df.withColumn("comment", regexp_replace("comment", "\\/\*", ""))

# # Remove numbers
# df = df.withColumn("comment", regexp_replace("comment", "\\d+", ""))

# Tokenize the comments
tokenizer = Tokenizer(inputCol="comment", outputCol="words")
df_words = tokenizer.transform(df)

# Remove stopwords
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
df_filtered = remover.transform(df_words)

# Explode the words into separate rows
df_exploded = df_filtered.withColumn("word", explode(col("filtered_words")))

# Filter for bots and non-bots, then count word frequencies
bot_word_counts = df_exploded.filter(df_exploded["bot"] == True).groupBy("word").count().orderBy("count", ascending=False)
non_bot_word_counts = df_exploded.filter(df_exploded["bot"] == False).groupBy("word").count().orderBy("count", ascending=False)

# Show top 20 words for bots and non-bots
print("Top 20 words used by bots:")
bot_word_counts.show(20)

print("Top 20 words used by regular users:")
non_bot_word_counts.show(20)

Top 20 words used by bots:
+---------------+-----+
|           word|count|
+---------------+-----+
|               |18179|
|             /*| 5763|
|             */| 5754|
|       category| 5509|
|              -| 4244|
|           page| 3649|
|         within| 3643|
|       included| 3643|
|        pages]]| 3641|
|        removed| 3421|
|       {{wpbs}}| 3017|
|      {{wpbs}}.| 2993|
|         rating| 2258|
|          added| 2159|
|         adding| 2063|
|              1| 1937|
|geograph.org.uk| 1746|
|  {{wikiproject| 1695|
|          vital| 1637|
|    articles]]:| 1637|
+---------------+-----+
only showing top 20 rows

Top 20 words used by regular users:
+--------------------+-----+
|                word|count|
+--------------------+-----+
|                    | 9666|
|                  /*| 6833|
|                  */| 6598|
|            category| 6327|
|               added| 5674|
|             removed| 1383|
|             tangkis| 1311|
|                bulu| 1311|
|wbsetdescriptio